In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

In [3]:
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done


In [4]:
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

In [5]:
# Start a SparkSession
import findspark
findspark.init()

In [6]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-11-24 00:32:32--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.59MB/s    in 0.6s    

2022-11-24 00:32:33 (1.59 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [8]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...|2015-08-31 00:00:00|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...| 

### Create DataFrames to match tables

In [9]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...|2015-08-31 00:00:00|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...| 

In [10]:
# Create the customers_table DataFrame
# customers_df = df.groupby("").agg({""}).withColumnRenamed("", "customer_count")
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   48670265|             1|
|   49103216|             2|
|    1131200|             1|
|   43076447|             2|
|   46261368|             1|
|    4883305|             5|
|   41192649|             1|
|   40985731|             7|
|   10437900|             2|
|   22245671|             1|
+-----------+--------------+
only showing top 10 rows



In [11]:
# Create the products_table DataFrame and drop duplicates. 
# products_df = df.select([]).drop_duplicates()
products_df = df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00CJ7IUI6|The Elder Scrolls...|
|B00DHF39KS|Wolfenstein: The ...|
|B00MUTAVH6|Under Night In-Bi...|
|B001AZSEUW|              Peggle|
|B00KVOVBGM|PlayStation 4 Con...|
|B00O9VGH4Y|USPRO&reg; Headph...|
|B004OQNZY4|Phineas and Ferb:...|
|B00ZLN980O|Donop seablue 2.4...|
|B002L8W5V6|Dotop Nintendo Ga...|
|B007AJZ5PY|Nyko Game Case fo...|
+----------+--------------------+
only showing top 10 rows



In [13]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
# review_id_df = df.select([, to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df = df.select(['review_id','customer_id','product_id','product_parent', to_date('review_date', 'yyyy-MM-dd').alias("review_date")])
review_id_df.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RTIS3L2M1F5SM|   12039526|B001CXYMFS|     737716809| 2015-08-31|
| R1ZV7R40OLHKD|    9636577|B00M920ND6|     569686175| 2015-08-31|
|R3BH071QLH8QMC|    2331478|B0029CSOD2|      98937668| 2015-08-31|
|R127K9NTSXA2YH|   52495923|B00GOOSV98|      23143350| 2015-08-31|
|R32ZWUXDJPW27Q|   14533949|B00Y074JOM|     821342511| 2015-08-31|
|R3AQQ4YUKJWBA6|    2377552|B002UBI6W6|     328764615| 2015-08-31|
|R2F0POU5K6F73F|   17521011|B008XHCLFO|      24234603| 2015-08-31|
|R3VNR804HYSMR6|   19676307|B00BRA9R6A|     682267517| 2015-08-31|
| R3GZTM72WA2QH|     224068|B009EPWJLA|     435241890| 2015-08-31|
| RNQOY62705W1K|   48467989|B0000AV7GB|     256572651| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [15]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = df.select(['review_id','star_rating','vine','helpful_votes','total_votes','verified_purchase'])
vine_df.show(10)

+--------------+-----------+----+-------------+-----------+-----------------+
|     review_id|star_rating|vine|helpful_votes|total_votes|verified_purchase|
+--------------+-----------+----+-------------+-----------+-----------------+
| RTIS3L2M1F5SM|          5|   N|            0|          0|                Y|
| R1ZV7R40OLHKD|          5|   N|            0|          0|                Y|
|R3BH071QLH8QMC|          1|   N|            0|          1|                Y|
|R127K9NTSXA2YH|          3|   N|            0|          0|                Y|
|R32ZWUXDJPW27Q|          4|   N|            0|          0|                Y|
|R3AQQ4YUKJWBA6|          1|   N|            0|          0|                Y|
|R2F0POU5K6F73F|          5|   N|            0|          0|                Y|
|R3VNR804HYSMR6|          5|   N|            0|          0|                Y|
| R3GZTM72WA2QH|          5|   N|            0|          0|                Y|
| RNQOY62705W1K|          4|   N|            0|          0|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [16]:
from getpass import getpass
password = getpass("Enter PgAdmin password")

Enter PgAdmin password··········


In [17]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://myclassdb.cbs5kmrpfy8t.us-west-2.rds.amazonaws.com:5432/AmazonReview"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

In [18]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [19]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [20]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [21]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [22]:
vine_df.write.csv('vine_df.csv')

Deliverable 2

In [23]:
#The data is filtered to create a DataFrame or table where there are 20 or more total votes (5 pt)
analysis_df = vine_df.filter(vine_df.total_votes >= 20)
analysis_df.show(10)

+--------------+-----------+----+-------------+-----------+-----------------+
|     review_id|star_rating|vine|helpful_votes|total_votes|verified_purchase|
+--------------+-----------+----+-------------+-----------+-----------------+
| R4PKAZRQJJX14|          1|   N|           21|         34|                N|
|R2CI0Y288CC7E2|          1|   N|           21|         35|                Y|
|R127WEQY2FM1T3|          1|   N|          147|        175|                Y|
|R3EZ0EPYLDA34S|          1|   N|           14|         31|                Y|
|R2FJ94555FZH32|          2|   N|           55|         60|                N|
|R1U3AR67RE273L|          1|   N|           51|         65|                Y|
|R3PZOXA5X1U8KW|          4|   N|           31|         36|                N|
| R6KTC1OPIOIIG|          2|   N|           19|         34|                Y|
|R36O341WWXXKNP|          5|   N|           28|         31|                N|
|R3GSK9MM8DNOYI|          1|   N|            4|         32|     

In [24]:
#The data is filtered to create a DataFrame or table where the percentage of helpful_votes is equal to or greater than 50% (5 pt)
percentage_df = analysis_df.filter((analysis_df.helpful_votes / analysis_df.total_votes) >= 0.5)
percentage_df.show(10)

+--------------+-----------+----+-------------+-----------+-----------------+
|     review_id|star_rating|vine|helpful_votes|total_votes|verified_purchase|
+--------------+-----------+----+-------------+-----------+-----------------+
| R4PKAZRQJJX14|          1|   N|           21|         34|                N|
|R2CI0Y288CC7E2|          1|   N|           21|         35|                Y|
|R127WEQY2FM1T3|          1|   N|          147|        175|                Y|
|R2FJ94555FZH32|          2|   N|           55|         60|                N|
|R1U3AR67RE273L|          1|   N|           51|         65|                Y|
|R3PZOXA5X1U8KW|          4|   N|           31|         36|                N|
| R6KTC1OPIOIIG|          2|   N|           19|         34|                Y|
|R36O341WWXXKNP|          5|   N|           28|         31|                N|
|R10LZVBLQHBVJ0|          2|   N|          151|        198|                N|
|R1VR5GLGY1GE7N|          1|   N|           49|         51|     

In [25]:
#The data is filtered to create a DataFrame or table where there is a Vine review (5 pt)
paid_df = percentage_df.filter(percentage_df.vine == 'Y')
paid_df.show(5)

+--------------+-----------+----+-------------+-----------+-----------------+
|     review_id|star_rating|vine|helpful_votes|total_votes|verified_purchase|
+--------------+-----------+----+-------------+-----------+-----------------+
|R3KKUSGFZWSUIY|          5|   Y|           56|         63|                N|
|R10FO5UKKVZBK2|          3|   Y|           23|         23|                N|
| RM4KSGEOR7MU1|          5|   Y|           19|         24|                N|
| RG7VRMYLEXD23|          4|   Y|           22|         26|                N|
|R11O4YSCPSNL6L|          3|   Y|           20|         26|                N|
+--------------+-----------+----+-------------+-----------+-----------------+
only showing top 5 rows



In [27]:
#The data is filtered to create a DataFrame or table where there isn’t a Vine review (5 pt)
not_paid_df = percentage_df.filter(percentage_df.vine == 'N')
not_paid_df.show(5)

+--------------+-----------+----+-------------+-----------+-----------------+
|     review_id|star_rating|vine|helpful_votes|total_votes|verified_purchase|
+--------------+-----------+----+-------------+-----------+-----------------+
| R4PKAZRQJJX14|          1|   N|           21|         34|                N|
|R2CI0Y288CC7E2|          1|   N|           21|         35|                Y|
|R127WEQY2FM1T3|          1|   N|          147|        175|                Y|
|R2FJ94555FZH32|          2|   N|           55|         60|                N|
|R1U3AR67RE273L|          1|   N|           51|         65|                Y|
+--------------+-----------+----+-------------+-----------+-----------------+
only showing top 5 rows



In [28]:
total_paid_reviews = paid_df.count()
total_paid_reviews

94

In [29]:
paid_five_star_reviews = paid_df.filter(paid_df.star_rating == 5).count()
paid_five_star_reviews

48

In [30]:
paid_five_star_percent = (paid_five_star_reviews / total_paid_reviews) * 100
paid_five_star_percent

51.06382978723404

In [31]:
total_unpaid_reviews = not_paid_df.count()
total_unpaid_reviews

40471

In [33]:
unpaid_five_star_reviews = not_paid_df.filter(not_paid_df.star_rating == 5).count()
unpaid_five_star_reviews

15663

In [34]:
unpaid_five_star_percent = (unpaid_five_star_reviews / total_unpaid_reviews) * 100
unpaid_five_star_percent

38.701786464381904